In [127]:
import csv
from datetime import datetime
import requests
from bs4 import BeautifulSoup

## 1) Prepare URL

In [93]:
def get_url(position, location):
    """Generate url from position and location"""
    template = 'https://www.indeed.com/jobs?q={}&l={}'
    position = position.replace(' ', '+')
    location = location.replace(' ', '+')
    url = template.format(position, location)
    return url

### 1a) Get Clinical Informatics Jobs

In [94]:
url = get_url('Clinical Informatics', 'USA')
print(url)

https://www.indeed.com/jobs?q=Clinical+Informatics&l=USA


### 1b) Get Health Informatics Jobs

In [95]:
url = get_url('Health Informatics', 'USA')
print(url)

https://www.indeed.com/jobs?q=Health+Informatics&l=USA


## 2) Search Jobs

In [99]:
response = requests.get(url)
#print(response)
soup = BeautifulSoup(response.text, 'html.parser')
#print(soup)#job_seen_beacon
cards = soup.find_all('div', 'job_seen_beacon')
#print(cards)
print(len(cards))

15


## 3) Get job post

In [7]:
def get_record(card):
    """Extract job data from a single record"""
    job_header= card.table.tbody.tr.td
    job_title = job_header.div.h2.a.span.get('title')
    job_link= 'https://www.indeed.com'+job_header.div.h2.a.get('href')

    try:
      job_company=job_header.find('span','companyName').text.strip()
    except:
      print("No company name found for job "+job_title)

    try:
      job_ratings=job_header.find('span','ratingsDisplay').text.strip()
    except:
      print("No job ratings found for job "+job_title)
      job_ratings=""

    try:
      job_location=job_header.find('div','companyLocation').text.strip()
    except:
      print("No job location found for job "+job_title)

    try:
      job_date=card.find('span','date').text.strip().replace("Posted", "").replace("EmployerActive", "").strip()
      print(job_date)
    except:
      print("No job date found for job "+job_title)

    try:
      job_snippets=card.find_all('div','attribute_snippet')
      print(len(job_snippets))
    except:
      #print("No job location found for job "+job_title)
      job_snippets=""

    if job_snippets!="":
        try:      
          job_salary=job_snippets.find('svg',{'aria-label':'Salary'})
        except:
          #print("No job salary found for job "+job_title)
          job_salary=""
        try:      
          job_type=job_snippets.find('svg',{'aria-label':'Job type'})
        except:
          #print("No job type found for job "+job_title)
          job_type=""
        try:      
          job_shift=job_snippets.find('svg',{'aria-label':'Shift'})
        except:
          #print("No job shift found for job "+job_title)
          job_shift=""
    else:
        job_salary=""
        job_type=""
        job_shift=""

    try:      
        job_brief=card.find('div','job-snippet').ul.li.text.strip().replace('\n', ' ')
    except:
        job_brief=""
    
    print(job_brief)

    collect_date = datetime.today().strftime('%Y-%m-%d')     
    record = (job_title, job_link, job_company, job_location, job_date, collect_date, job_brief, job_ratings,job_salary, job_type, job_shift)
    return record

## 4) Get all jobs by scraping the next pages of the search

In [ ]:
records=[]
url=get_url('Health Informatics', 'USA') 
#url=get_url('Clinical Informatics', 'USA') # Uncomment and comment the above line to retrieve the Health Informatics Jobs
for i in range(67):
    print(url)
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    cards = soup.find_all('div', 'job_seen_beacon')

    for card in cards:
        record = get_record(card)
        if record in records:
            continue
        else:
            records.append(record)
    try: 
        url = 'https://www.indeed.com' + soup.find('a', {'aria-label': 'Next'}).get('href')
    except AttributeError:
        print('search results error in iter '+str(i))
        break

In [26]:
print(len(records))

831


## 5) Save job posts in file

In [27]:
# save the job data
with open('results.csv', 'w', newline='', encoding='utf-8') as f:
    writer = csv.writer(f)
    writer.writerow(['JobTitle', 'JobUrl','Company', 'Location', 'PostDate', 'ExtractDate', 'Summary','Job_ratings', 'Salary', 'JobType','JobShift'])
    writer.writerows(records)

## 6) Getting jobs briefing: title, URL, summary, location, etc from the search result of the previous steps

In [6]:
from selenium import webdriver
import pandas as pd
jobs = pd.read_csv('results.csv')
titles = jobs['JobTitle'].tolist()
urls = jobs['JobUrl'].tolist()
company = jobs['Company'].tolist()
location= jobs['Company'].tolist()
PostDate= jobs['PostDate'].tolist()
ExtractDate= jobs['ExtractDate'].tolist()
Summary= jobs['Summary'].tolist()
Job_ratings= jobs['Job_ratings'].tolist()
Salary= jobs['Salary'].tolist()
JobType= jobs['JobType'].tolist()
JobShift= jobs['JobShift'].tolist()

In [17]:
# Installing the web driver for web browser
pip install webdriver-manager

Note: you may need to restart the kernel to use updated packages.


In [18]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

driver = webdriver.Chrome(ChromeDriverManager().install())

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 101.0.4951
[WDM] - Get LATEST chromedriver version for 101.0.4951 google-chrome
[WDM] - There is no [mac64] chromedriver for browser 101.0.4951 in cache
[WDM] - About to download new driver from https://chromedriver.storage.googleapis.com/101.0.4951.41/chromedriver_mac64.zip
[WDM] - Driver has been saved in cache [/Users/aelnosho/.wdm/drivers/chromedriver/mac64/101.0.4951.41]
/var/folders/7v/g_t4898131v7f09flvsqg0v80000gn/T/ipykernel_1115/2160715986.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


In [100]:
import os
# The below is the path of the web browser driver on the machine used for this research.
# To reuse, change the below file location to the location of the web browser driver on your local machine
os.path.exists('/Users/aelnosho/.wdm/drivers/chromedriver/mac64/101.0.4951.41')

True

## 7) Getting jobs details by opening each job URL

In [ ]:
import sys
import time
from datetime import datetime
import requests
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
count=0
jobsdesc=[]
for l in urls:
    url = l
    # initiating the webdriver. Parameter includes the path of the webdriver.
    driver = webdriver.Chrome('/Users/aelnosho/.wdm/drivers/chromedriver/mac64/101.0.4951.41/chromedriver') 
    try:
        driver.get(url) 

        # this is just to ensure that the page is loaded
        time.sleep(5) 

        html = driver.page_source
        # this renders the JS code and stores all
        # of the information in static HTML code.

        # Now, we could simply apply bs4 to html variable
        soup = BeautifulSoup(html, "html.parser")

        try:
            salary = soup.find('div', {'id' : 'salaryGuide'}).ul.text
            Salary[count]=salary
        except:
            print("No salary found for job ")
        jobDescription = soup.find('div', {'id' : 'jobDescriptionText'}).text
        #print(jobDescription)
        jobsdesc.append(jobDescription)
    except:
        print("Description of job number "+str(count)+" was not retrieved")
        jobsdesc.append("")
    count=count+1
    
driver.close() # closing the webdriver

In [33]:
print(len(jobsdesc))

571


In [8]:
## Checking for error loading the job descriptions pages detail
diff=len(urls)-count
print(diff)
print(count)

0
831


## 8) load remaining jobs that did not load in previous point due to error loading

In [ ]:
for i in range(count, len(jobsdesc)):
    url=urls[u]
    print(url)
    driver = webdriver.Chrome('/Users/aelnosho/.wdm/drivers/chromedriver/mac64/101.0.4951.41/chromedriver') 
    driver.get(url) 

    # this is just to ensure that the page is loaded
    time.sleep(5) 

    html = driver.page_source
    # this renders the JS code and stores all
    # of the information in static HTML code.

    # Now, we could simply apply bs4 to html variable
    soup = BeautifulSoup(html, "html.parser")
    
    try:
        salary = soup.find('div', {'id' : 'salaryGuide'}).ul.text
        Salary[count]=salary
    except:
        print("No salary found for job ")
        
    jobDescription = soup.find('div', {'id' : 'jobDescriptionText'}).text
    jobsdesc.append(jobDescription)
    count=count+1

## 9) save jobs with job description into csv

In [29]:
df_updated = pd.DataFrame({'Job_titles':titles,'Job_URLS':urls,'Company':company,'location':location,'PostDate':PostDate,'ExtractDate':ExtractDate,'Summary':Summary,'Job_ratings':Job_ratings,'Job_description':jobsdesc,'Salary':Salary,'JobType':JobType,'JobShift':JobShift})
df_updated.to_csv('JobsDetails.csv')

## 10) Analyze jobs

### 10A) Clean the job description

In [126]:
import pandas as pd
df_updated = pd.read_csv(r'JobDetailsAnalysis-HI.csv')

In [128]:
from html.parser import HTMLParser
import re
from nltk.corpus import wordnet
from nltk.stem.wordnet import WordNetLemmatizer

class MLStripper(HTMLParser):
    def __init__(self):
        self.reset()
        self.fed = []
    def handle_data(self, d):
        self.fed.append(d)
    def get_data(self):
        return ''.join(self.fed)

def strip_tags(html):
    s = MLStripper()
    s.feed(html)
    return s.get_data()

def cleanURLEmail(strip):
    # extract emails
    match = re.findall(r'[\w\.-]+@[\w\.-]+', strip)
    temp = strip
    for m in match:
        temp = strip.replace(m, '', 100)
        strip = temp
    match3 = re.findall(r'^https?:\/\/.*[\r\n]*', strip)
    for m in match3:
        temp = strip.replace(m, '', 100)
        strip = temp
    return strip


#Remove Stop words, punctuation, and extract lemmatization, and stemming.
def clean(doc,stop,exclude):
    lemma = WordNetLemmatizer()
    stop_free = " ".join([i for i in str(doc).lower().split() if i not in stop])
    no_punc = map(lambda x: re.sub("[^a-z]", " ", x), stop_free)
    punc_free = "".join(ch for ch in no_punc if ch not in exclude)
    normalized=""
    stemmed = ""
    for word in punc_free.split():
        try:
            lWord=lemma.lemmatize(word)
        except:
            continue
        normalized=normalized+lWord+" "
    stop_free2 = " ".join([i for i in normalized.lower().split() if i not in stop])
    return stop_free2

In [129]:
from selenium import webdriver
import time
import csv
import pandas as pd
#import simplejson
from nltk import ngrams
import datetime
import re
import string
import nltk
import requests
from nltk.corpus import stopwords
import unicodedata



driver = webdriver.Chrome('/Users/aelnosho/.wdm/drivers/chromedriver/mac64/101.0.4951.41/chromedriver') 
us_state_abbrev = {
    'Alabama': 'AL',
    'Alaska': 'AK',
    'Arizona': 'AZ',
    'Arkansas': 'AR',
    'California': 'CA',
    'Colorado': 'CO',
    'Connecticut': 'CT',
    'Delaware': 'DE',
    'Florida': 'FL',
    'Georgia': 'GA',
    'Hawaii': 'HI',
    'Idaho': 'ID',
    'Illinois': 'IL',
    'Indiana': 'IN',
    'Iowa': 'IA',
    'Kansas': 'KS',
    'Kentucky': 'KY',
    'Louisiana': 'LA',
    'Maine': 'ME',
    'Maryland': 'MD',
    'Massachusetts': 'MA',
    'Michigan': 'MI',
    'Minnesota': 'MN',
    'Mississippi': 'MS',
    'Missouri': 'MO',
    'Montana': 'MT',
    'Nebraska': 'NE',
    'Nevada': 'NV',
    'New Hampshire': 'NH',
    'New Jersey': 'NJ',
    'New Mexico': 'NM',
    'New York': 'NY',
    'North Carolina': 'NC',
    'North Dakota': 'ND',
    'Ohio': 'OH',
    'Oklahoma': 'OK',
    'Oregon': 'OR',
    'Pennsylvania': 'PA',
    'Rhode Island': 'RI',
    'South Carolina': 'SC',
    'South Dakota': 'SD',
    'Tennessee': 'TN',
    'Texas': 'TX',
    'Utah': 'UT',
    'Vermont': 'VT',
    'Virginia': 'VA',
    'Washington': 'WA',
    'West Virginia': 'WV',
    'Wisconsin': 'WI',
    'Wyoming': 'WY',
}
statesBSC={}
statesMaster={}
statesCertifications={}
states={}
for k, v in us_state_abbrev.items():
    statesBSC[v]=0
    statesMaster[v] = 0
    statesCertifications[v] = 0
    states[v]=0


def cleanJobDescription(description):
    doc_complete=[]
    doc_clean=[]
    tList=[]
    exclude = set(string.punctuation)
    stop = set(stopwords.words('english'))
    lines = open("stop3").read().splitlines()
    for word in lines:
        #print(word)
        stop.add(word)
    myStop = ["a", "about", "above", "above", "across", "after", "afterwards", "again", "against", "all",
                 "almost", "alone", "along", "already", "also", "although", "always", "am", "among", "amongst",
                 "amoungst", "amount", "an", "and", "another", "any", "anyhow", "anyone", "anything", "anyway",
                 "anywhere", "are", "around", "as", "at", "back", "be", "became", "because", "become", "becomes",
                 "becoming", "been", "before", "beforehand", "behind", "being", "below", "beside", "besides",
                 "between", "beyond", "bill", "both", "bottom", "but", "by", "call", "can", "cannot", "cant", "co",
                 "con", "could", "couldnt", "cry", "de", "describe", "detail", "do", "done", "down", "due",
                 "during", "each", "eg", "eight", "either", "eleven", "else", "elsewhere", "empty", "enough", "etc",
                 "even", "ever", "every", "everyone", "everything", "everywhere", "except", "few", "fifteen",
                 "fify", "fill", "find", "fire", "first", "five", "for", "former", "formerly", "forty", "found",
                 "four", "from", "front", "full", "further", "get", "give", "go", "had", "has", "hasnt", "have",
                 "he", "hence", "her", "here", "hereafter", "hereby", "herein", "hereupon", "hers", "herself",
                 "him", "himself", "his", "how", "however", "hundred", "ie", "if", "in", "inc", "indeed",
                 "interest", "into", "itself", "keep", "last", "latter", "latterly", "least",
                 "less", "ltd", "made", "many", "may", "me", "meanwhile", "might", "mill", "mine", "more",
                 "moreover", "most", "mostly", "move", "much", "must", "my", "myself", "name", "namely", "neither",
                 "never", "nevertheless", "next", "nine", "no", "nobody", "none", "noone", "nor", "not", "nothing",
                 "now", "nowhere", "of", "off", "often", "on", "once", "one", "only", "onto", "or", "other",
                 "others", "otherwise", "our", "ours", "ourselves", "out", "over", "own", "part", "per", "perhaps",
                 "please", "put", "rather", "re", "same", "see", "seem", "seemed", "seeming", "seems", "serious",
                 "several", "she", "should", "show", "side", "since", "sincere", "six", "sixty", "so", "some",
                 "somehow", "someone", "something", "sometime", "sometimes", "somewhere", "still", "such", "system",
                 "take", "ten", "than", "that", "the", "their", "them", "themselves", "then", "thence", "there",
                 "thereafter", "thereby", "therefore", "therein", "thereupon", "these", "they", "thickv", "thin",
                 "third", "this", "those", "though", "three", "through", "throughout", "thru", "thus", "to",
                 "together", "too", "top", "toward", "towards", "twelve", "twenty", "two", "un", "under", "until",
                 "up", "upon", "us", "very", "via", "was", "we", "well", "were", "what", "whatever", "when",
                 "whence", "whenever", "where", "whereafter", "whereas", "whereby", "wherein", "whereupon",
                 "wherever", "whether", "which", "while", "whither", "who", "whoever", "whole", "whom", "whose",
                 "why", "will", "with", "within", "without", "would", "yet", "you", "your", "yours", "yourself",
                 "yourselves", "the", 'http', 'https', '://', 'www', 'com', '8800', '...', '....','//','/','..', 'yep', '.).', '](#', '.:).',
                '++..', 'github', 'etc', 'also', 'org', 'gee', 'let', 'know', 'ever',
                'vcntr', 'falseamount', 'isig']
    for word in myStop:
        stop.add(word)
        
    #strip HTML tags from tweet
    temp=""
    #strip = strip_tags(description)
    #strip1 = cleanURLEmail(description)
    strip1 = str(description).replace(".","")
    strip2 = clean(strip1,stop,'')
    return(strip2)



/var/folders/7v/g_t4898131v7f09flvsqg0v80000gn/T/ipykernel_812/3880222035.py:17: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('/Users/aelnosho/.wdm/drivers/chromedriver/mac64/101.0.4951.41/chromedriver')


In [130]:
df_updated['cleanDesc']= df_updated.apply(lambda x : cleanJobDescription(x['Job_description']),axis=1)

### 10b) Get skills, certificates, training, and degrees

In [132]:
# Funcation to flag the job if it required or prefer a professional certification or a university degree
def checkDegrees(description,degreeCheck):
    result=0
    graduateKeywords=['mba','msc', 'md', 'master', 'phd', 'doctorate']
    bachelorKeywords = ['college', 'bachelor', 'bsc']
    degrees=['mba','msc', 'md', 'master', 'phd', 'doctorate','college', 'bachelor', 'bsc']
    if degreeCheck in str(description):
        result= 1
    elif degreeCheck == 'bachelor':
        for word in bachelorKeywords:
            if word in str(description):
                result = 1
                break
    elif degreeCheck == 'graduate':
        for word in graduateKeywords:
            if word in str(description):
                result = 1
                break
    elif degreeCheck == 'certifications':
        if "ertified" in str(description) or "ertificat" in str(description):
                result = 1
    elif degreeCheck == 'otherDegree':
        count=0
        if 'degree' in str(description):
            for word in degrees:
                if word in str(description):
                    result = 0
                    break
                count=count+1
                if count>8: 
                    result = 1
    return result

In [133]:
# Function that extracts the certifications, skills, programming languages, data visualization tools required 
# or prefered in the job description
# check to exclude certification if has any of the following degrees=['mba','msc', 'md', 'master', 'phd', 'doctorate','college', 'bachelor', 'bsc']
def getText(description, search):
    count = 0
    degree=0
    # For professional certifications exclude, any academic certification 
    degrees=['mba','msc', 'md', 'master', 'phd', 'doctorate','college', 'bachelor', 'bsc']
    try:
        TexttotSet=set()
        words=description.lower().split()
        count=0
        cert=0
        for w in words:
            temp=""
            if (search=="certifications" and ("ertified" in w or "ertificat" in w)) or (search=="skills" and "skill" in w) or (search=="trainings" and "train" in w)or (search=="degrees" and "degree" in w) or (search=="license" and "license" in w) or (search=="visualization" and "visualization" in w) or (search=="programming" and "progamming" in w):
                degree=0
                counter=4
                i=count-1
                tempQueue=[]
                while(i>-1 and counter>0):
                    if words[i] in degrees:
                        degree=1
                        break
                    tempQueue.append(words[i])
                    counter-=1
                    i-=1
                if degree==0:
                    while(len(tempQueue)>0):
                        temp=tempQueue.pop(0)+" "+temp
                counter=1
                while (degree==0 and counter<5 and counter+count<len(words)):
                    if words[count+counter] in degrees:
                        degree=1
                        break
                    temp=temp+" "+words[count+counter]
                    counter+=1
                if degree==0:
                    TexttotSet.add(temp)
            count+=1
        
        # Extract the certification or skill name that could consists of one word, two words, three words...upto six words
        s=""
        for t in TexttotSet:
            Text=str(t)
            #Bigrams
            bigramsText=""
            n = 2
            n_grams = ngrams(Text.split(), n)
            for grams in n_grams:
                temp=""
                temp=grams[0]+"_"+grams[1]
                bigramsText=bigramsText+" "+temp

            #trigrams
            trigramsText = ""
            n = 3
            n_grams = ngrams(Text.split(), n)
            for grams in n_grams:
                temp=grams[0]+"_"+grams[1]+"_"+grams[2]
                trigramsText = trigramsText + " " + temp

            fourgramsText = ""
            n = 4
            n_grams = ngrams(Text.split(), n)
            for grams in n_grams:
                temp=""
                temp=grams[0]+"_"+grams[1]+"_"+grams[2]+"_"+grams[3]
                fourgramsText = fourgramsText + " " + temp

            fivegramsText=""
            n = 5
            n_grams = ngrams(Text.split(), n)
            for grams in n_grams:
                temp=""
                temp=grams[0]+"_"+grams[1]+"_"+grams[2]+"_"+grams[3]+"_"+grams[4]
                fivegramsText = fivegramsText + " " + temp

            sixgramsText=""
            n = 6
            n_grams = ngrams(Text.split(), n)
            for grams in n_grams:
                temp=""
                temp=grams[0]+"_"+grams[1]+"_"+grams[2]+"_"+grams[3]+"_"+grams[4]+"_"+grams[5]
                sixgramsText = sixgramsText + " " + temp

            s=s+Text+" "+bigramsText+" "+trigramsText+" "+fourgramsText+" "+fivegramsText+" "+sixgramsText+" "
            return str(s.strip())
    except Exception as e:
        print(e)
        

In [134]:
df_updated['certifications']= df_updated.apply(lambda x : getText(x['cleanDesc'],'certifications'),axis=1)
df_updated['skills']= df_updated.apply(lambda x : getText(x['cleanDesc'],'skills'),axis=1)
df_updated['trainings']= df_updated.apply(lambda x : getText(x['cleanDesc'],'trainings'),axis=1)
df_updated['degrees']= df_updated.apply(lambda x : getText(x['cleanDesc'],'degrees'),axis=1)
df_updated['license']= df_updated.apply(lambda x : getText(x['cleanDesc'],'license'),axis=1)
df_updated['programming']= df_updated.apply(lambda x : getText(x['cleanDesc'],'programming'),axis=1)
df_updated['visualization']= df_updated.apply(lambda x : getText(x['cleanDesc'],'visualization'),axis=1)

In [135]:
df_updated['bachelor']=df_updated.apply(lambda x : checkDegrees(x['Job_description'],'bachelor'),axis=1)
df_updated['graduate']=df_updated.apply(lambda x : checkDegrees(x['Job_description'],'graduate'),axis=1)
df_updated['otherDegree']=df_updated.apply(lambda x : checkDegrees(x['Job_description'],'otherDegree'),axis=1)

In [136]:
def checkCollege(bachelor,graduate):
    if bachelor ==1 or graduate==1:
        return 1
    else:
        return 0
df_updated['college']=df_updated.apply(lambda x : checkCollege(x['bachelor'],x['graduate']),axis=1)

In [ ]:
# Jobs with undergraduate or graduate degrees requested
df_updated[df_updated['college']==1]

In [ ]:
# Jobs with College graduate degrees requested
df_updated[df_updated['graduate']==1]

In [ ]:
# Jobs that requests bachelors degrees
df_updated[df_updated['bachelor']==1]

In [138]:
# Flag the job if a certification is requested
def checkCertificationLenght(certification):
    if certification is None:
        return 0
    return len(certification)
df_updated['certificationFlag']=df_updated.apply(lambda x : checkCertificationLenght(x['certifications']),axis=1)

In [90]:
df_updated[df_updated['certificationFlag']>0].to_csv('certifications.csv')

# The reason why 40% of the jobs listed prefering or requiring certification while 
# the jobs with specifically mentioned certifications are almost 10% to 15% is that
# most of the jobs either mentioned that certification is required or prefered without specifying like:
# "Certification relevant to training," "Industry designation certifications," "Industry-focused Certifications,"
# or "Speciality certification is required", etc. Also, some jobs mentioned "Certification support" as part 
# of the professional development benefits in the job to attract candidates focusing on continous improvement
 

In [27]:
# Checking the location of the jobs by state

# Checking jobs that have the flexibility of working remotely
def checkRemote(location):
    if 'remote' in location.lower():
        return 1
    else:
        return 0
df_updated['remote']=df_updated.apply(lambda x : checkRemote(x['location']),axis=1)

In [28]:
statesAbbrev=list(us_state_abbrev.values())
statesAb=[]
for st in statesAbbrev:
    statesAb.append(" "+st)
states=list(us_state_abbrev.keys())

In [29]:
def checkState(location):
    for st in statesAb:
        if st in location:
            return st.strip()
    for st in states:
        if st in location:
            return us_state_abbrev[st]
    return '0'
df_updated['states']=df_updated.apply(lambda x : checkState(x['location']),axis=1)

In [30]:
df_updated['states'].value_counts().to_csv('States.csv')

### N.B. 1/4 of the jobs have a remote working flexibility

In [ ]:
df_updated[df_updated['remote']==1]

### Checking job flexibility with college degree requirement (no correlation found)

In [81]:
df_updated['remote'].corr(df_updated['college'])

-0.07909216195943626

### Checking the skills, certifications, data visualization tools in highest demand

In [142]:
def getHighDemandJobDescription(df,demand):
    def preprocess(text):
        tokens = []
        tokenizer = nltk.tokenize.RegexpTokenizer(r"[a-z]+")
        for token in tokenizer.tokenize(str(text)):
            tokens.append(token)
        return tokens
    if demand=='skills':
        List=df['skills'].tolist()
    elif demand=='certifications':
        List=df['certifications'].tolist()
    elif demand=='trainings':
        List=df['trainings'].tolist()
    elif demand=='license':
        List=df['license'].tolist()
    elif demand=='programming':
        List=df['programming'].tolist()
    elif demand=='visualization':
        List=df['visualization'].tolist()
    else:
        List=df['degrees'].tolist()
    processed = list(map(preprocess, List))
    fdist = nltk.FreqDist([token for doc in processed for token in doc])
    print(fdist.tabulate(1000))

In [ ]:
# Get high demand skills
getHighDemandJobDescription(df_updated,'skills')
# Get high demand certifications
getHighDemandJobDescription(df_updated,'certifications')
# Top university degrees listed 
getHighDemandJobDescription(df_updated,'degrees')

In [ ]:
# Get high demand data visualization and programming tools
import numpy as np
df=df_updated
df['visualization'].replace('', np.nan).dropna(inplace=True)
print(df)
getHighDemandJobDescription(df,'visualization')

In [ ]:
maskSkill1 = df_updated['cleanDesc'].str.contains(r'SQL', na=True)
df_updated[maskSkill1]

# The number of jobs requesting each skill:
# Analytics and Problem Sovling 623
# Communication skills 337
# Oral Communication 238
# Project Management 199
# Statistics 198
# Interpersonal skills 172
# Critical thinking 67

# The top mentioned tools
# Excel 514
# SQL 150

# Tool listed tools

#mask6 = df_updated['cleanDesc'].str.contains(r'js', na=True) & df_updated['cleanDesc'].str.contains(r'certificat', na=True)
#mask6 = df_updated['cleanDesc'].str.contains(r'critical thinking', na=True) & df_updated['graduate']==1
#mask6 = df_updated['cleanDesc'].str.contains(r'java', na=True)
#mask6 = df_updated['Job_description'].str.contains(r' R ', na=True)
#mask6 = df_updated['cleanDesc'].str.contains(r'power bi', na=True)
df_updated[mask6]
# Python 74
# Tableau 73
# Java 29
# R 24
# SPSS 14
# Power BI 13


In [159]:
# List the top listed keywords for certifications in spreadsheet
CertificationsWording_list = df_updated["certifications"].tolist()
certificationDictionary={}
for certificationWording in CertificationsWording_list:
    if certificationWording is None:
        continue
    l= certificationWording.split()
    for w in l:
        if w in certificationDictionary:
            certificationDictionary[w]+=1
        else:
            certificationDictionary[w]=1

In [163]:
with open('certificatesKeywords.csv','w') as file:
    line="word,count"
    file.write(line)
    file.write('\n')
    for k in certificationDictionary:
        line=k+","+str(certificationDictionary[k])
        file.write(line)
        file.write('\n')

In [ ]:
# checking in how many jobs certain certifications was listed
mask3 = df_updated['cleanDesc'].str.contains(r'cpc', na=True)
df_updated[mask3]

In [109]:
# The average indeed ratings of Institutions that listed the jobs
print(df_updated.agg({"Job_ratings": ["mean"]}))
print(len(df_updated.Company.unique()))

      Job_ratings
mean     3.730213
592


In [103]:
# Save the results of analysis in csv
df_updated.to_csv('JobDetailsAnalysis.csv')